## LIBRARIES

In [1]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import random
from operator import itemgetter    
import time

## Extra Modules to limit Code Clutter

In [2]:
def load_folder(folder):
    image_list = [] # creating empty list
    for file in os.listdir(folder): # iterating through folder
        img = cv2.imread(os.path.join(folder, file)) # reading each image in forlder
        if img is not None: # only if image read correctly will it be appended
            image_list.append(img)
    return image_list

def convert_image_bw(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img_gray.copy()

def show_image_window(img, title):
    while True:
        cv2.imshow(title, img)
        if cv2.waitKey(1) == 27: # if user hits ESC key --> window is exited
            break 
    cv2.destroyAllWindows()
    
def show_image_plt(img, title):
    plt.imshow(img)
    plt.title(title)

def show_row_image_plt(images):
    fig = plt.figure()
    rows = 1
    columns = len(images)
    i = 1
    for img in images:
        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
        plt.title("i")
    
def feature_detection(img1, img2):
    img1 = convert_image_bw(img1)
    img2 = convert_image_bw(img2)
    my_SIFT_instance = cv2.SIFT_create()
    kp1, desc1 = my_SIFT_instance.detectAndCompute(img1, None) # returns list of keypoints and an array of 128xkp
    kp2, desc2 = my_SIFT_instance.detectAndCompute(img2, None) # setting mask field to None
    
    bf_matcher = cv2.BFMatcher()  # returns the best match
    matches = bf_matcher.knnMatch(desc1, desc2, k=2)  # returns the k best matches
    good_match = []
    for m, n in matches:
        if m.distance < 0.6*n.distance: # using each matches euclidean distance
            good_match.append(m) 
    
    points_l = np.float32([kp1[i.queryIdx].pt for i in good_match]).reshape(-1, 1, 2) # queryIdx = This attribute gives us the index of the descriptor in the list of img1 descriptors
    points_r = np.float32([kp2[i.trainIdx].pt for i in good_match]).reshape(-1, 1, 2) # trainIdx = This attribute gives us the index of the descriptor in the list of img2 descriptors
    return points_l, points_r


def determine_count(src_img, BM_img): # given keypoints determine if position of BM in L or R
    count = 0
    for i in range(len(src_img)):
        if BM_img[i][0][0] < src_img[i][0][0]:
            count += 1
    return count

def remove_element(my_list, target):
    my_list.remove(target)
    return my_list

def remove_border_list(list_of_img):
    list_copy = list_of_img.copy()
    for i in range(len(list_copy)):
        bw = cv2.cvtColor(list_copy[i],cv2.COLOR_BGR2GRAY)
        _,threshold = cv2.threshold(bw,0,255,cv2.THRESH_BINARY)
        contours,_ = cv2.findContours(threshold,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        x,y,w,h = cv2.boundingRect(contours[0])
        list_copy[i] = list_copy[i][y:y+h,x:x+w]
    return list_copy

def remove_border(img):
    img_copy = img.copy()
    bw = cv2.cvtColor(img_copy,cv2.COLOR_BGR2GRAY)
    _,threshold = cv2.threshold(bw,0,255,cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(threshold,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    x,y,w,h = cv2.boundingRect(contours[0])
    img_copy = img_copy[y:y+h,x:x+w]
    return img_copy

## STEP 01 MATCH FEATURES: Feature Extraction and Feature Correspondence

In [3]:
def match_features(folder): # STEP 01: The first step is to extract features from the images, and automatically establish feature correspondences between image pairs.
    start_time = time.time()
    sorted_image_list = []
    images_to_compare = [i for i in range(len(folder))] # initialize list of images to compare w current
    current_image = 0
    sorted_image_list.append(current_image) # add the first image to the list
    images_to_compare = remove_element(images_to_compare, current_image) # remove it from the compare list and find the closest R image
    count_list = []
    while True:
        # print("Images Left To Compare: {}".format(images_to_compare))
        max_count = 0
        best_match = 0
        if len(images_to_compare) == 1: # check if last image in list is to the L of initial image
            pi, pj = feature_detection(folder[images_to_compare[0]], folder[sorted_image_list[0]])
            count = determine_count(pi, pj)
            pi2, pj2 = feature_detection(folder[sorted_image_list[0]], folder[sorted_image_list[1]])
            count2 = determine_count(pi2, pj2)
            if count >= count2*0.6:
                sorted_image_list.insert(0, images_to_compare[0])
            images_to_compare = remove_element(images_to_compare, images_to_compare[0])
        if len(images_to_compare) == 0:
            print("Sorted Order: {}".format(sorted_image_list))
            break
        for j in images_to_compare:
            # print("COMPARING WITH: ", j)
            pi, pj = feature_detection(folder[current_image], folder[j])
            count = determine_count(pi, pj)
            if count > max_count:
                max_count = count
                best_match = j
                # print("BM UPDATED")
        sorted_image_list.append(best_match)
        print("Sorted Order: {}".format(sorted_image_list))
        images_to_compare = remove_element(images_to_compare, best_match)
        current_image = best_match
        count_list.append(max_count)
        print("MATCH METRIC OUTPUT: {}".format(count_list))
    print("--- MATCH FEATURES EXECUTION TIME: %s seconds ---" % (time.time() - start_time))
    return sorted_image_list

def determine_image_pairs(sorted_list, folder):
    start_time = time.time()
    center_img = int(len(sorted_list)/2)
    leftward_pairs = []
    rightward_pairs = []
    for i in range(len(sorted_list)): # want to warp <-<-<-<-=->->->->
        if i < center_img:
            current_index = i 
            warp_onto = i+1
            leftward_pairs.append((sorted_list[current_index], sorted_list[warp_onto]))
        if i > center_img and i < len(sorted_list)-1:
            current_index = i
            warp_onto = i+1
            rightward_pairs.append((sorted_list[current_index], sorted_list[warp_onto]))
    print("--- DETERMINING IMAGE PAIRS TIME: %s seconds ---" % (time.time() - start_time))
    return leftward_pairs, rightward_pairs, center_img

## STEP02 ESTIMATE TRANSFORMATION: Calculating Transformations Between Matched Features
## STEP 03 MERGE IMAGES: Applying Geometric and Radiometric Transformations

In [4]:
def inital_pair_warp_R(rightward_pairs, folder):
    warped_img = []
    list_of_pair_warps = []
    list_of_warps = []
    for i in range(len(rightward_pairs)): # from center->0
        print("Pair: {}".format(i))
        right_img = folder[rightward_pairs[i][1]]
        left_img = folder[rightward_pairs[i][0]]
        pi, pj = feature_detection(right_img, left_img) # project left image onto image to right
        M, _ = cv2.findHomography(pi, pj, cv2.RANSAC, 5.0)
        h1, w1, _1 = right_img.shape
        h2, w2, _2 = left_img.shape
        width = w1+w2
        height = h2+h1
        warped_img = cv2.warpPerspective(right_img, M, (width, height))
        warped_img[0:h2, 0:w2] = left_img
        list_of_warps.append(warped_img)
    print(len(list_of_warps))
    list_of_pair_warps = remove_border_list(list_of_warps)
    return list_of_pair_warps

def warping_to_right_pair(right_img, left_img):
    warped_img = []
    pi, pj = feature_detection(right_img, left_img) # project left image onto image to right
    M, _ = cv2.findHomography(pi, pj, cv2.RANSAC, 5.0)
    h1, w1, _1 = right_img.shape
    h2, w2, _2 = left_img.shape
    width = w1+w2
    height = h2+h1
    warped_img = cv2.warpPerspective(right_img, M, (width, height))
    warped_img[0:h2, 0:w2] = left_img
    warped_img = remove_border(warped_img)
    return warped_img

def recursive_warping_R(rightward_pairs, folder):
    r_copy = rightward_pairs.copy()
    input_list = []
    output_list = inital_pair_warp_R(r_copy, folder)
    final_warp = []
    while True:
        if len(output_list) == 1: # if the final output is only one image of all combined images we are done
            break
        print("OUTPUT LEN:{}".format(len(output_list)))
        input_list = output_list.copy()
        output_list.clear()
        while True:
            print("INPUT LEN:{}".format(len(input_list))) 
            if len(input_list) == 1: # if size is odd just add last image to end
                print("List Is ODD")
                output_list.append(input_list[0])
                input_list.clear()
            if len(input_list) == 0:
                break
            warped_img = []
            right_img = input_list[1]
            left_img = input_list[0]
            input_list.remove(left_img)
            input_list.remove(right_img)
            warped_img = warping_to_right_pair(right_img, left_img) 
            output_list.append(warped_img)
    print("DONE")
    final_warp = output_list[0]
    return final_warp
# -----------------------------------------------------------------------
# Flipping all LEFTWARD_PAIR IMAGES TO APPLY RIGHTWARD STITCHING AND FLIPPING RESULT
def flip_image2(img):
    print("Flipping")
    img_height, img_width, img_channel = img.shape
    hor_flipped_img = np.zeros((img_height,img_width,3), np.uint8) # initializing empty images with same width, height, channel and dtype
    for i in range(img_height):
        for j in range(img_width):
            for k in range(img_channel):
                hor_flipped_img[i][j][k] = img[i][img_width-j-1][k] # horizontally flipped image has a reveresed order of columns in its array values
    return hor_flipped_img

def flip_image(img):
    img = cv2.flip(img, 1)
    return img

# Apply initial warps from leftward_pairs list and flip all images and send to recursive warp
def inital_pair_warp_L(leftward_pairs, folder):
    warped_img = []
    list_of_pair_warps = []
    list_of_warps = []
    for i in range(len(leftward_pairs)): # from center->0
        print("RIGHT: {}, LEFT: {}".format(leftward_pairs[i][1], leftward_pairs[i][0]))
        right_img = flip_image(folder[leftward_pairs[i][0]])
        left_img = flip_image(folder[leftward_pairs[i][1]])
        print("flipped")
        pi, pj = feature_detection(right_img, left_img) # project right image onto image to left
        M, _ = cv2.findHomography(pi, pj, cv2.RANSAC, 5.0)
        h1, w1, _1 = right_img.shape
        h2, w2, _2 = left_img.shape
        width = w1+w2
        height = h2+h1
        warped_img = cv2.warpPerspective(right_img, M, (width, height)) # warp R IMG IN TERMS OF L IMG
        warped_img[0:h2, 0:w2] = left_img # left image is maintained at beginning of image, warped R_img is "added" to end 
        list_of_warps.append(warped_img)
        print(len(list_of_warps))
    list_of_pair_warps = remove_border_list(list_of_warps) # remove border around images
    initial_pair_list = list_of_pair_warps.copy()
    return initial_pair_list

def recursive_warping_L(leftward_pairs, folder):
    l_copy = leftward_pairs.copy()
    input_list = []
    output_list = inital_pair_warp_L(l_copy , folder) # warping all the inital pairs together -> convert pair list to images
    final_warp = []
    while True:
        if len(output_list) == 1: # if the final output is only one image of all combined images we are done
            break
        else:
            print("OUTPUT LEN:{}".format(len(output_list)))
            input_list = output_list.copy()
            output_list.clear()
            while True:
                print("INPUT LEN:{}".format(len(input_list))) 
                if len(input_list) == 1: # if size is odd just add last image to end
                    print("List Is ODD")
                    output_list.append(input_list[0])
                    input_list.clear()
                if len(input_list) == 0:
                    break
                else:
                    warped_img = []
                    right_img = input_list[1]
                    left_img = input_list[0]
                    input_list.remove(left_img)
                    input_list.remove(right_img)
                    warped_img = warping_to_right_pair(right_img, left_img) 
                    output_list.append(warped_img)
        plt.imshow(warped_img)
        plt.show()
    print("DONE")
    final_warp = flip_image(output_list[0])
    return final_warp

In [5]:
# Load the Folder
def AutoStitch_Panorama(folder_name):
    start_time = time.time()
    folder_images = load_folder(folder_name)
    img_list = match_features(folder_images)
    leftward_pairs, rightward_pairs, center_img = determine_image_pairs(img_list, folder_images)
    R_final = recursive_warping_R(rightward_pairs, folder_images)
    L_final = recursive_warping_L(leftward_pairs, folder_images)
    FINAL_PANO = warping_to_right_pair(R_final, L_final)
    print("--- FULL AUTOSTITCH EXECUTION TIME: %s seconds ---" % (time.time() - start_time))
    return FINAL_PANO, R_final, L_final
    

In [6]:
img_1, R_final, L_final = AutoStitch_Panorama('office2')

Sorted Order: [0, 1]
MATCH METRIC OUTPUT: [542]
Sorted Order: [0, 1, 2]
MATCH METRIC OUTPUT: [542, 467]
Sorted Order: [0, 1, 2, 3]
MATCH METRIC OUTPUT: [542, 467, 390]
Sorted Order: [0, 1, 2, 3, 4]
MATCH METRIC OUTPUT: [542, 467, 390, 121]
Sorted Order: [0, 1, 2, 3, 4, 5]
MATCH METRIC OUTPUT: [542, 467, 390, 121, 173]
Sorted Order: [0, 1, 2, 3, 4, 5, 6]
MATCH METRIC OUTPUT: [542, 467, 390, 121, 173, 146]
Sorted Order: [0, 1, 2, 3, 4, 5, 6, 7]
MATCH METRIC OUTPUT: [542, 467, 390, 121, 173, 146, 139]
Sorted Order: [0, 1, 2, 3, 4, 5, 6, 7, 8]
MATCH METRIC OUTPUT: [542, 467, 390, 121, 173, 146, 139, 159]
Sorted Order: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
MATCH METRIC OUTPUT: [542, 467, 390, 121, 173, 146, 139, 159, 166]
Sorted Order: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
MATCH METRIC OUTPUT: [542, 467, 390, 121, 173, 146, 139, 159, 166, 350]
Sorted Order: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
MATCH METRIC OUTPUT: [542, 467, 390, 121, 173, 146, 139, 159, 166, 350, 273]
Sorted Order: [0, 1, 2, 3, 4,

error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'cv::findHomography'


In [ ]:
cv2.imwrite('Resulting_Composite_Img_01.jpg', img_1)

In [ ]:
cv2.imwrite('Resulting_Left_Pano_01.jpg', L_final)

In [ ]:
cv2.imwrite('Resulting_Right_Pano_01.jpg', R_final)

In [ ]:
img_2, R_final2, L_final2 = AutoStitch_Panorama('StJames')

In [ ]:
cv2.imwrite('Resulting_Composite_Img_02.jpg', img_2)

In [ ]:
img_3, R_final3, L_final3 = AutoStitch_Panorama('WLH')

In [ ]:
cv2.imwrite('Resulting_Composite_Img_03.jpg', img_3)